In [1]:
import lammps
import copy
import numpy as np
import random
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output

In [2]:
# Constants
k_B = 8.617333262145e-5  # Boltzmann constant in eV/K

# Input Parameters
species_dict = {'Pt': [1, 195.084, 0.5], 'Au': [2, 196.967, 0.5]} # Dictionary containing each species and their type/mass/composition
phases_dict = {'alpha_1': ['fcc', 4.02, 1], 'alpha_2': ['fcc', 4.02, 2]} # Dictionary containing each phase and their lattice type/parameter
potential_file = 'PtAu.eam.alloy' # Path to potential file
unit_cells = 3
iterations = 10000
equilib = 5000

start_temp = 1000
final_temp = 1600
resolution = 100

In [3]:
# Initialize one lammps instance for each phase
def init_instances(n, p, s, pot):
    unit_cells = n
    phases_dict = p
    phases_count = len(phases_dict)
    species_dict = s
    species_count = len(species_dict)
    potential_file = pot

    instances = []
    for phase, values in phases_dict.items():
        lmp = lammps.lammps(cmdargs=['-echo', 'none', '-log', 'none', '-nocite', '-screen', 'none']) # initialize instance

        lmp.command(f'variable  n  equal {unit_cells}') 
        lmp.command('units  metal') 
        lmp.command('boundary  p p p') 
        lmp.command('atom_style atomic') 
        lmp.command(f'lattice {values[0]} {values[1]}') # lattice lattice_type param
        lmp.command('atom_modify map array sort 0 0.0') 
        lmp.command('region  box block 0 ${n} 0 ${n} 0 ${n}') 
        lmp.command(f'create_box   {species_count} box') # atomic species
        for species, data in species_dict.items():
            lmp.command(f'mass {data[0]} {data[1]}') # mass atom_type mass
        lmp.command('pair_style  eam/alloy') # eam potential style
        lmp.command(f'pair_coeff * * {potential_file} Pt Au') # potential file
        lmp.command(f'create_atoms {values[2]} box')
        lmp.command(f'dump init_dump all xyz 1 initial_positions_{phase}.xyz')
        lmp.command('undump init_dump')
        lmp.command('compute pe all pe')
        lmp.command('run 0')
        
        instances.append(lmp)

    return instances, phases_count, species_count

In [4]:
def init_fracs(l, pc, sc):
    instances = l
    phases_count = pc
    species_count = sc
    
    A_c = np.zeros((phases_count, species_count))
    A = np.zeros((phases_count, species_count))
    all_atom_count = 0
    
    for ind, inst in enumerate(instances):
        total_atoms = inst.get_natoms()
        all_atom_count += total_atoms
        num_atom_types = inst.extract_global('ntypes', 0)

        # Initialize a list to store the count of each atom type
        atom_counts = np.zeros(num_atom_types)
        
        # Loop through all atoms and count each type
        for i in range(total_atoms):
            atom_type = inst.gather_atoms("type", 0, 1)[i]
            atom_counts[atom_type - 1] += 1  # LAMMPS atom types are 1-based
        A_c[ind] = atom_counts
        A[ind] = atom_counts / total_atoms
        
    A_c = A_c.transpose().astype(int)
    A = A.transpose()
    X_0 = np.vstack(np.sum(A_c, axis=1) / all_atom_count)

    try:
        molfrac = np.linalg.solve(A, X_0)
    except np.linalg.LinAlgError:
        print('ERROR: INITIAL MASS BALANCE MATRIX IS SINGULAR')  # If solving fails, throw error

    pot_eng = calc_pe(instances, molfrac)
    
    return A_c, A, X_0, molfrac, pot_eng

In [5]:
def soft_perturb(l, ac, a, sc):
    instances = l
    A_c = ac
    A = a
    species_count = sc
    
    # Flip move, choose instance
    rand_inst = np.random.randint(len(instances))
    inst = instances[rand_inst]

    # Choose a random atom to flip
    natoms = inst.get_natoms()
    rand_atom = np.random.randint(1, natoms + 1)

    # Find the random atom's type, choose a random type to flip it to
    init_type = inst.gather_atoms("type", 0, 1)[rand_atom - 1]
    new_type = np.random.randint(1, species_count + 1)
    while init_type == new_type:
        new_type = np.random.randint(1, species_count + 1)

    # Add an atom of the new type, subtract the atom of the old type
    A_c[init_type - 1, rand_inst] -= 1
    A_c[new_type - 1, rand_inst] += 1
    inst.command(f'set atom {rand_atom} type {new_type}')
    #inst.command('run 0')
    
    #print(f'set atom {rand_atom} in instance {rand_inst} type {new_type} from {init_type}')
    A[:, rand_inst] = A_c[:, rand_inst] / natoms
    log = [rand_inst, rand_atom, new_type, init_type]
    
    return A_c, A, log, instances

In [6]:
def calc_pe(p, m):
    instances = p
    molfrac = m
    pot_eng = 0
    
    for ind, inst in enumerate(instances):
        inst.command('run 0')
        pot_eng += molfrac[ind, 0]*inst.extract_compute("pe", 0, 0)
    
    return pot_eng

In [7]:
def iterate(p, ac, a, x0, sc, it, m, pe, sd, pd, t, eq):
    instances = p
    A_c = ac
    A = a
    X_0 = x0
    species_count = sc
    phases_count = len(instances)
    iterations = it
    molfrac = m
    pot_eng = pe
    species_dict = sd
    phases_dict = pd
    temp = t
    beta = 1 / (k_B * temp) # Thermodynamic Beta
    equilib = eq
    
    tol = 1e-10  # Define a tolerance for determining singular matrices

    # Plotting 
    # Initialize the plot
    plt.ion()  # Turn on interactive mode
    plt.switch_backend('Qt5Agg')

    # Overall molar fraction plot
    fig, ax = plt.subplots()
    ax.set_xlim(0, iterations)
    ax.set_ylim(0, 1)
    xdata = []
    ydata = [[] for _ in range(phases_count)]

    # Phase composition plot
    fig2, axs2 = plt.subplots(phases_count, 1, figsize=(10, 15))
    if phases_count == 1:
        axs2 = [axs2]  # Ensure axs2 is iterable even for a single subplot
    for ax2 in axs2:
        ax2.set_xlim(0, 100)
        ax2.set_ylim(0, 1)
    ydata2 = [[[] for _ in range(species_count)] for _ in range(phases_count)]


    
    # Main simulation loop
    for step in range(iterations):
        pot_eng_old, A_c_old, A_old = copy.deepcopy(pot_eng), copy.deepcopy(A_c), copy.deepcopy(A)
        
        # Perturb accordingly
        loglist = []
        A_c, A, log, instances = soft_perturb(instances, A_c, A, species_count)
        loglist.append(log)
        
        # Initialize molfrac with a default value
        molfrac = np.zeros_like(X_0)
        
        det = np.linalg.det(A_c)
        
        # Check initial conditions before entering the while loop
        if np.abs(det) >= tol:
            try:
                molfrac = np.linalg.solve(A, X_0)
            except np.linalg.LinAlgError:
                pass  # Continue to perturb if solving fails
        
        while np.abs(det) < tol or np.any(molfrac < 0):
            A_c, A, log, instances = soft_perturb(instances, A_c, A, species_count)
            loglist.append(log)
            det = np.linalg.det(A_c)
            
            if np.abs(det) >= tol:
                try:
                    molfrac = np.linalg.solve(A, X_0)
                except np.linalg.LinAlgError:
                    continue  # If solving fails, continue to perturb

        # Calculate new potential energy
        pot_eng = calc_pe(instances, molfrac)
        delta_pe = pot_eng - pot_eng_old

        # Calculate acceptance criterion
        prob = np.min([1, np.exp(- phases_count * (delta_pe) * beta)])

        # Reject accordingly
        if np.random.uniform(0.0, 1.0) > prob:
            A_c, A, pot_eng = A_c_old, A_old, pot_eng_old
            for undo in reversed(loglist):
                inst = instances[undo[0]]
                inst.command(f'set atom {undo[1]} type {undo[3]}')
                #inst.command('run 0')
                
        # More plot stuff
        xdata.append(step)
        
        # Overall molar fraction plot
        for col in range(phases_count):
            for j in range(species_count):
                ydata2[col][j].append(A[j][col])
                
        # Phase composition plot
        for j in range(phases_count):
            ydata[j].append(molfrac[j][0])
        
        # Plot stacked area plot
        ax.clear()
        ax.stackplot(xdata, *ydata, alpha=0.5, labels=[f'{list(phases_dict.keys())[j]}' for j in range(phases_count)])
        ax.set_xlim(0, step+0.01)
        ax.set_title(f'Overall Phase Fractions (T = {temp}K, n = {unit_cells})')
        ax.legend(loc='upper right')

        for col, ax2 in enumerate(axs2):
            ax2.clear()
            ax2.set_xlim(0, step+0.01)
            ax2.set_ylim(0, 1)
            ax2.set_title(f'{list(phases_dict.keys())[col]} Phase Composition (T = {temp}K, n = {unit_cells})')
        
            # Plot stacked area plot with transparency
            ax2.stackplot(xdata, *ydata2[col], alpha=0.5, labels=[f'{list(species_dict.keys())[j]}' for j in range(phases_count)])
            ax2.legend(loc='upper right')

        # Draw the updated plot
        fig.canvas.draw()
        fig.canvas.flush_events()

        # Draw the updated plots
        fig2.canvas.draw()
        fig2.canvas.flush_events()

    
    plt.ioff()  # Turn off interactive mode
    fig.savefig(f'Plots/PtAu_frac_{temp}.png')
    fig2.savefig(f'Plots/PtAu_comp_{temp}.png')
    plt.close('all')

    return np.array(ydata)[:, equilib:], np.array(ydata2)[:, :, equilib:]

    

In [8]:
def float_to_percent(value):
    # Multiply by 100 to convert to percentage and format with five significant figures
    percent_value = value * 100
    formatted_percent = f"{percent_value:.5g}%"
    
    return formatted_percent

In [9]:
def temp_scan(u, pd, sd, pf, s, f, r, eq):
    unit_cell = u
    phases_dict = pd
    species_dict = sd
    potential_file = pf
    start_temp = s
    final_temp = f
    resolution = r
    equilib = eq

    phase_data = []
    species_data = []
    temp_data = []
    phase_stds = []
    ovr_stds = []

    for temp in range(start_temp, final_temp, resolution):
        print(f'NOW RUNNING TEMPERATURE = {int(temp)}K')
        
        instances, phases_count, species_count = init_instances(unit_cells, phases_dict, species_dict, potential_file)
        A_c, A, X_0, molfrac, pot_eng = init_fracs(instances, phases_count, species_count)
        ydata_sliced, ydata2_sliced = iterate(instances, A_c, A, X_0, species_count, iterations, molfrac, pot_eng, species_dict, phases_dict, temp, equilib)

        ovr_comp = np.mean(ydata_sliced, axis = 1)
        phase_comp = np.transpose(np.mean(ydata2_sliced, axis = 2))
        phase_std = np.std(ydata2_sliced, axis = 2)
        ovr_std = np.std(ydata_sliced, axis=1)

        phase_list = []
        species_list = []
        for phase in range(phases_count):

            comp_val = ovr_comp[phase]
            phase_list.append(comp_val)
            
            pct = float_to_percent(comp_val)
            species_str = ''

            specs = []
            for species in range(species_count):
                
                # Extract values
                species_float = phase_comp[species, phase]
                specs.append(species_float)
                
                species_pct = float_to_percent(phase_comp[species, phase])
                species_str += str(species_pct) + ' ' + str(list(species_dict.keys())[species])
                if species < species_count - 1:
                    species_str += '; '
            species_list.append(specs)      
            print(f'{list(phases_dict.keys())[phase]} Phase Percent: {pct}, Lattice Type: {phases_dict[list(phases_dict.keys())[0]][0]}, \nComposition: {species_str}')
        phase_data.append(phase_list)
        species_data.append(species_list)
        temp_data.append(temp)
        phase_stds.append(phase_std)
        ovr_stds.append(ovr_std)
    return phase_data, species_data, temp_data, phase_stds, ovr_stds

In [ ]:
phase_data, species_data, temp_data, phase_stds, ovr_stds = temp_scan(unit_cells, phases_dict, species_dict, potential_file, start_temp, final_temp, resolution, equilib)

NOW RUNNING TEMPERATURE = 1000K
alpha_1 Phase Percent: 41.374%, Lattice Type: fcc, 
Composition: 78.751% Pt; 21.249% Au
alpha_2 Phase Percent: 58.626%, Lattice Type: fcc, 
Composition: 29.748% Pt; 70.252% Au
NOW RUNNING TEMPERATURE = 1100K


KeyboardInterrupt: 

In [ ]:
# pt = np.array(species_data)[:,:,0]
# pt_stds = np.array(phase_stds/np.sqrt(iterations - equilib))[:,:,0]
# print(temp_data)
# print(pt)
# print(pt_stds)
print(ovr_stds)
